In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from scipy import stats
import cv2 as cv
import os
from calibracion import puntos
from tabulate import tabulate
#list_img: resive una lista de imagenes a graficar, la lista debe estar en formato vector fila
#filas: nuermo de filas del subplot
#colum: numero de columnas del subplot
#Nota para graficar se debe formar una regilla mas grande o igual al numero de imagenes, pero no menor.
#list_titulos: no es  necesario entregarla. permite colocarle entregar un titulo para cada imagen. 
def graficar(list_img,filas,colum,list_titulos=[]):
    numero=len(list_img)
    titulos=len(list_titulos)
    if titulos!=numero:
        titulos=0
        print("numero de titulos incorrecto")
    plt.figure(figsize=(40,20))
    for i in range(numero):
        plt.subplot(filas,colum,i+1)
        plt.imshow(list_img[i])
        if titulos>0 : 
            plt.title(list_titulos[i],fontsize=40)
           
    plt.show()

# capturar imagenes.
1. las imagenes del sensor RGB para la calibracion se guardaran unicamente en escala de grises.
2. el nombre de todas las imagenes tiene el siguiente formato sensor_numero.png. sensor=gray para las imagenes del sensor RGB, sensor=depth para el sensor de profundidad. un numero se colocara para diferenciar imagenes tomadas del mismo  sensor. el numero tambien permitira agrupar las imagenes de los dos sensores en pares que contengan el mismo numero. 

In [ ]:
from ventana import ventana
def obtener_img_calibracion(path,nueva_carpeta):
    # initialize interface
    interface=ventana("presionar f: para guardar img; c para salir")
    # comprobar carpeta de almacenamiento
    #path='/home/estufab/Escritorio/flujo de bagazo/codigo/img_calibracion'
    if  not(os.path.isdir("{}/{}/".format(path,nueva_carpeta))):
        os.mkdir("{}/{}".format(path,nueva_carpeta))
    #actualizar path
    path+=nueva_carpeta
    # Initialize the depth device
    openni2.initialize()
    dev = openni2.Device.open_any()

    # Start the depth stream
    depth_stream = dev.create_depth_stream()
    depth_stream.start()
    depth_stream.set_video_mode(c_api.OniVideoMode(pixelFormat = c_api.OniPixelFormat.ONI_PIXEL_FORMAT_DEPTH_100_UM, resolutionX = 640, resolutionY = 480, fps = 30))
    rgb_stream = dev.create_color_stream()
    rgb_stream.set_video_mode(c_api.OniVideoMode(pixelFormat=c_api.OniPixelFormat.ONI_PIXEL_FORMAT_RGB888, resolutionX= 640, resolutionY = 480, fps = 30))
    rgb_stream.start()

    def get_gray():                                                                  
        bgr   = np.frombuffer(rgb_stream.read_frame().get_buffer_as_uint16(),dtype=np.uint8).reshape(480,640,3)
        gray   = cv2.cvtColor(bgr,cv2.COLOR_BGR2RGB)
        return gray
    x=0
    while True:
        # Grab a new depth frame


        img_gray = get_gray()    
        depth = np.frombuffer(depth_stream.read_frame().get_buffer_as_uint16(),dtype=np.uint16)
        depth.shape = (1, 480, 640)
        depth = np.concatenate((depth, depth, depth), axis=0)
        depth = np.swapaxes(depth, 0, 2)  #las columnas las vuelvo filas
        depth = np.swapaxes(depth, 0, 1) # las filas las vuelvo columnas 
        #pseudo color
        depth_copy=cv2.cvtColor(depth, cv2.COLOR_BGR2GRAY)/256
        depth_color = cv2.applyColorMap(depth_copy.astype(np.uint8), cv2.COLORMAP_JET)#COLORMAP_JET##COLORMAP_PINK

        #visualizacion
        interface.visualizar([img_gray,depth_color],img_gray.shape[0:2],1,2,["RGB","DEPTH"])


        key = cv2.waitKey(1) & 0xFF
        # If the 'c' key is pressed, break the while loopc
        if key == ord("c"):
                       break
        if key == ord("f"):
            x+=1
            path_gray=os.path.join(path, 'gray_{}.png'.format(x))
            path_matrix_depth=os.path.join(path, 'gray_{}.png'.format(x))
            path_depth_16=os.path.join(path, 'depth_{}.png'.format(x))
            cv2.imwrite(path_depth_16,depth)
            np.save(path_matrix_depth,depth)
            ret=cv2.imwrite(path_gray,img_gray)
            print(ret)
            print ("end-start")

    openni2.unload()
    cv2.destroyAllWindows()

In [ ]:
from calibracion import obtener_img_calibracion
path='/home/estufab4/Desktop/flujo de bagazo/codigo/img_calibracion/'
nueva_carpeta="prb3"
a=obtener_img_calibracion(path,nueva_carpeta)

# cambiar nombre imgs

In [ ]:
def cambiar_nombre(path,nombre,nuevo_nombre):
    #path="/home/estufab/Escritorio/flujo de bagazo/codigo/img_calibracion/RGB5/"
    images = glob.glob("{}{}*.png".format(path,nombre))
    for name in images:
        index=name.find(nombre)
        n=len(nombre)
        nuevo=name[:index]+nuevo_nombre+name[index+n:]
        print(nuevo)
        os.rename(name,nuevo)
cambiar_nombre("/home/estufab/Escritorio/flujo de bagazo/codigo/img_calibracion/union/","deph","depth")
        

# flip  conjunto de imagenes 

In [ ]:
#path: directorio absoluto del lugar donde se almacenan las imagenes.
#tipo: 0 para elegir flip vertical, 1 para elegir flip horizontal
def flip(path,tipo):
    #path="/home/estufab/Escritorio/flujo de bagazo/codigo/img_calibracion/RGB5/"
    images = glob.glob("{}*.png".format(path))
    for fname in images:
        img =cv2.imread(fname)
        img=np.flip(img,axis=tipo)
        cv2.imwrite(fname,img)
flip("/home/estufab/Escritorio/flujo de bagazo/codigo/img_calibracion/profundidad5/",1)

# funciones

In [ ]:
def puntos_objeto(d,nf,nc):
   #  d distancia entre punto medio de circulos/ la multiplicación no afecta la matriz intrinseca
    # preparar puntos de objeto, como (0,0,0), (1,0,0), (2,0,0)...., (6,5,0) 
    objp = np.zeros((nf*nc,3), np.float32)
    objp[:,:2] = np.mgrid[0:nc,0:nf].T.reshape(-1,2)# crea una grilla tipo chessboard

    # formar plano cordenado teniendo en cuenta las medidas del patron y su forma asimetrica
    objp=objp.reshape(nf,nc,3)
    for i in range(nf): #
      objp[i,:,1]=d*objp[i,:,1]/2# distancia entre filas
    #distancia entre circulos de una misma fila
      if i%2==0:
        objp[i,:,0]=d*objp[i,:,0]
      else:# filas desplazadas
        objp[i,:,0]=d*objp[i,:,0]+d/2
    objp=objp.reshape(nf*nc,3) 
    return objp
def calibracion(objpoints,imgpoints,shape_img):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints,shape_img[::-1],None,None)## las dimensiones que recive son de ancho por alto, por eso hay que invertir shape_img
    # refinamiento de la matriz intrinceca, ROI con recorte de  la imagen
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,shape_img[::-1],1)
    # calcular la inversa
    inv_mtx=np.linalg.inv(newcameramtx)
    total_error = 0 
    for i in range(len(objpoints)): 
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist) 
        error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    
        print("error por imagen:{}".format(error))
        total_error += error
    mean_error=total_error/len(objpoints)
    print ("error medio de reproyección en pixeles: ", mean_error )
    return mtx,newcameramtx,roi,inv_mtx, dist, rvecs, tvecs


# calibrar RGB

In [ ]:
nf=8
nc=5
d=20
objp=puntos_objeto(d,nf,nc)

puntos_rgb=puntos(nf,nc)
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_calibracion/RGB5/"
images = glob.glob("{}gray_*.png".format(path))
# Arrays para almacenar puntos de objeto y puntos de imagen de todas las imágenes.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
# extracción de puntos para todas las imagenes
for fname in images:
    #show paht and name of picture 
    print(fname)
    img =cv2.imread(fname)
    #img=np.flip(img,axis=0)
    ret,puntos_centro,_=puntos_rgb.extraer(img)
    print("hubo detección: {}".format(ret))
    if ret:
        # agregar puntos para la calibración
        objpoints.append(objp)
        imgpoints.append(puntos_centro)
        #dibujar puntos encontrados
        img_centers=puntos_rgb.dibujar(img,puntos_centro)
        #graficar resultados
        graficar(list_img=[img,img_centers],
                 filas=1,colum=2,list_titulos=["original","puntos centro"])

In [ ]:
#proceso de calibración
print(img.shape[:2])
mtx,newcameramtx,roi,inv_mtx, dist, rvecs, tvecs=calibracion(objpoints,imgpoints,img.shape[:2])

# crear clase calibracion_uv

In [ ]:
from calibracion import puntos,graficar
import glob
import cv2
import numpy as np
import os
from configparser import ConfigParser
# la clase solo realiza calibración de la informacion en las cordenadas uv de la imagen.
## la clase permite encontrar la matriz intrinseca y una versión optimizada de la misma. la clase tiene por separado las funciones de resultados y funsiones de visualización.
#la clase tiene una función que permite quitar la distorsion causada por los lentes, tambien quitar la distorción a todas las imagenes utilizadas para la calibración y las guarda con el titulo de la imagen original dentro de la subcarpeta distorción
# las imagenes deben estar guardadas con un nombre que posea el siguente formato.  cabecera_nombre.png. cabecera=gray indica si la imagen es RGB o cabecera=depth de profunidad. nombre puede adquirir cualquier valor.
# la clase puede calibrar informacion uv tanto para RGB como para profundidad. solo hay que cambiar el typo en el costructor. typo=0 RGB, tipo=1 profundidad.
# la clase permite guardar la informacion en urchivo .ini. se escoge este formato por dos razones. primero al almacenar la informacion con tiquetas, se  facilita  localizar cualquier dato. se puede editar las etiquetas facilmente, alterando solo la informacion requerita. esto ultimo permite agrupar toda la informacion de configuración en un solo archivo.
# al utilizar el tipo de archivo .ini toda la informacion se guarda en formato string. si bien habrá que reconvertirla a tipo array, se podra abrir el archivo cualquier editor de texto.
class calibracion_uv(puntos):
    

    #constructor
    #path:carpeta donde se encuentra la carpeta de imagenes. la dirección es absoluta. las imagenes deben estar guardadas con nombre de formato (tipo_indicativo.png). tipo podra ser gray o depht.
    #indicativo puede ser cualquier cosa para identificar a la imagen( un numero, una fecha etc..)
    #path: directorio absoluto donde se encentran las imagenes a guardar.
    #d: diametro de los circulos
    #nf: numero de filas
    #nc:numero de circulos por fila
    #tipo: indicador 0 Para RGB, 1 Para profundidad.
    def __init__(self,path,d,nf,nc,tipo=0):

        #constantes de intancia
        self.d=d# diametro de circulos
        self.nf=nf
        self.nc=nc
        self.tipo=tipo
        self.path=path
        #variables de instancia
        self.mtx=0# matriz intrinseca 
        self.mt_intrinseca=0# matriz intrinseca optimizada
        self.mt_inv=0# almacena la matriz inversa de la matriz intrinseca
        self.dist=0# almacena los coeficientes de distorción
        self.roi=0# almacena el area de interes, util para quitar la distorción 
        self.list_puntos_img=[]# cada elemento pertenece a una imagen diferente, cada elemento es una matriz con los puntos encontrados en la imagen.
        self.list_puntos_obj=[]# debe haber un elemento por cada elemento en list_puntos_img. todos los elementos en esta lista son iguales.
        self.list_nombres_img=[]# contiene una lista de la ruta y nombre  de las imagenes en la que los puntos fueron detectados. hay correspondencia uno a uno con lis_puntos_img
        self.list_error_img=[]# cada elemento tiene una correspondencia uno a uno con list_nombres_img. cada elementos es error promedio de proyección en pixeles de los puntos de una imagen. 
        self.error_promedio=0# contiene el error promedio de proyección, se calcula con respecto a todas las imagenes. En otras palabras es el promedio de los valores en list_error_img
        self.puntos_obj=self.puntos_objeto()
        self.img_shape=0#  contiene (alto,ancho)
        #funciones 
        puntos.__init__(self,nf,nc)# constructor de la clase puntos.
        self.obtener_puntos_img()
       
       
    def puntos_objeto(self):
        #d distancia entre punto medio de circulos/ la multiplicación no afecta la matriz intrinseca
        # preparar puntos de objeto, como (0,0,0), (1,0,0), (2,0,0)...., (6,5,0) 
        objp = np.zeros((self.nf*self.nc,3), np.float32)
        objp[:,:2] = np.mgrid[0:self.nc,0:self.nf].T.reshape(-1,2)# crea una grilla tipo chessboard

        # formar plano cordenado teniendo en cuenta las medidas del patron y su forma asimetrica
        objp=objp.reshape(self.nf,self.nc,3)
        for i in range(self.nf):
            # distancia entre filas
          objp[i,:,1]=self.d*objp[i,:,1]
        #distancia entre circulos de una misma fila
          if i%2==0:
            objp[i,:,0]=self.d*2*objp[i,:,0]
          else:# filas desplazadas
            objp[i,:,0]=self.d*2*objp[i,:,0]+self.d
        objp=objp.reshape(self.nf*self.nc,3) 
        return objp
    def obtener_puntos_img(self):
        if self.tipo==1:
            directorio = glob.glob("{}depth_*.png".format(self.path))
        else:
            directorio = glob.glob("{}gray_*.png".format(self.path))
        
   
        for name in directorio:
       
            #show paht and name of picture 
            print(name)
            img =cv2.imread(name)
            #img=np.flip(img,axis=1)
            ret,puntos_centro,_=self.extraer(img,self.tipo)
            print("hubo detección: {}".format(ret))
            if ret:
                # agregar puntos para la calibración
                self.list_puntos_img.append(puntos_centro)
                self.list_puntos_obj.append(self.puntos_obj)
                self.list_nombres_img.append(name)
            #actualizar las dimensiones de las imagenes
            self.shape_img=img.shape[:2]
        
        if len(self.list_puntos_img)==0:
                print("no hubo deteccion")
        else:
            
            # modificacion para impidir procesar mas de 100 img
            cnt=int(len(self.list_puntos_img)/100 +1)
            self.list_puntos_img=self.list_puntos_img[::cnt]
            self.list_puntos_obj=self.list_puntos_obj[::cnt]
            print("numero de imagenes a utilizar : ",len(self.list_puntos_img))
    def calibrar(self):
        if len(self.list_puntos_img)>0:
            print("calculando parametros...")
            ret, self.mtx, self.dist, rvecs, tvecs = cv2.calibrateCamera(self.list_puntos_obj,self.list_puntos_img,self.shape_img[::-1],None,None)## las dimensiones que recive son de ancho por alto, por eso hay que invertir gray.shape
            # refinamiento de la matriz intrinceca, ROI con recorte de  la imagen
            self.mt_intrinseca, self.roi=cv2.getOptimalNewCameraMatrix(self.mtx,self.dist,self.shape_img[::-1],1)
            self.roi=np.array(self.roi)# almacenar como un array numpy
            # calcular la inversa
            self.mt_inv=np.linalg.inv( self.mt_intrinseca)
            total_error = 0 
            for i in range(len(self.list_puntos_img)): 
                imgpoints2, _ = cv2.projectPoints(self.list_puntos_obj[i], rvecs[i], tvecs[i],self.mtx, self.dist) 
                error = cv2.norm(self.list_puntos_img[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
                self.list_error_img.append(error)
                total_error += error
            self.list_error_img=np.array(self.list_error_img)
            self.error_promedio=total_error/len(self.list_puntos_img)
            print ("error medio de reproyección en pixeles: ", self.error_promedio )
        else:
            print("no hubo ninguna detección")
    # cada elemento es un par de imagenes. una imagen original y una modificada.La imagen modificada tiene los puntos detectados.
    def dibujar_puntos(self):
        list_img=[]
        for i in range(len(self.list_nombres_img)):
            img =cv2.imread(self.list_nombres_img[i])
            #img=np.flip(img,axis=1)
            img_puntos=self.dibujar(img,self.list_puntos_img[i],0)
            list_img.append([img,img_puntos])
        return list_img
    #resive una imagen y devuelve una imagen sin distorsión teniendo en cuenta los paratros de la camara obtenidos durante la calibración 
    def distorsion(self,img):
        dst = cv2.undistort(img, self.mtx, self.dist, None, self.mt_intrinseca)
        return dst
    # distorsiona todas la imagenes utilizadas para la calibración y las guarda en una subcarpeta llamada distorsion
    def distorsion_imgs(self):
        #comprobar existencia de la carpeta distorsion
        if  not(os.path.isdir("{}/distorsion/".format(self.path))):
            os.mkdir("{}distorsion".format(self.path))
            
        for name in self.list_nombres_img:
            img =cv2.imread(name)
            img_dst=self.distorsion(img)
            # recortar imagen 
            #x,y,w,h = self.roi
            #img_dst = img_dst[y:y+h, x:x+w]
            if self.tipo:
                index=name.find("depth")# debido a que en el nombre de la imagen tiene el indicador de gray o depht. se puede extraer el nombre de la dirección y volver a guardar la imagen a la que se le quita el la distorción con el mismo nombre.
                cv2.imwrite("{}distorsion/{}".format(self.path,name[index:]),img_dst)
            else:
                index=name.find("gray")# debido a que en el nombre de la imagen tiene el indicador de gray o depht. se puede extraer el nombre de la dirección y volver a guardar la imagen a la que se le quita el la distorción con el mismo nombre.
                cv2.imwrite("{}distorsion/{}".format(self.path,name[index:]),img_dst)
   #la iformacion se guarda en un archivo .ini. que puede ser leido por cualquier editor de texto
    #toda la informacion se guarda como string, por eso habra que reconvertir cuando se guarden los datos
    #nombre: solo nombre de archivo sin extension
    #path: dirección absoluta del lugar donde se quiere guardar. si no se especifica el archivo se guardara en la carpeta donde se esta ejecutando el archivo .py
    def guardar(self,nombre,path=""):
        #crear objeto
        config = ConfigParser()
        #revisar existencia del archivo
        ret=config.read('{}{}.ini'.format(path,nombre))
        if ret: #verificar si el objeto esta vacio
            ret=True
        else:
            ret=False
        #configurar tipo de información rgb o de profundidad
        if self.tipo:
            tipo="depth"
        else:
            tipo="rgb"
        print(config)
        #toda la informacion se convierte a vector fila para facilitar luego su lectura.   
        config['calibracion_uv_{}'.format(tipo)] = {"mtx": self.mtx.reshape(-1),
                                                    "mt_intrinseca":self.mt_intrinseca.reshape(-1),
                                                    "mt_inversa": self.mt_inv.reshape(-1),
                                                    "coef_dist":self.dist.reshape(-1),
                                                    "roi":self.roi.reshape(-1),
                                                    "error":self.error_promedio
                                   }
        
        #guardar archivo
        with open('{}{}.ini'.format(path,nombre), 'w') as configfile:
            config.write(configfile)
        return ret
        

                
        

# utilizar clase calibración_uv para RGB

In [ ]:

nf=9
nc=6
d=5
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_calibracion/calibracion2/d5_110/"
calibracion_rgb=calibracion_uv(path,d,nf,nc,0)

# visualización de las imagenes detectadas y sus puntos

In [ ]:
list_img=calibracion_rgb.dibujar_puntos()
for i in range(len(list_img)):
    directorio_img=calibracion_rgb.list_nombres_img[i]
    index=directorio_img.find("gray")
    nombre_img=directorio_img[index:]
    graficar(list_img[i],1,2,[nombre_img,"imagen con puntos"])

# si todas las imagenes son correctas se procede a utilizarlas para calibrar

In [ ]:
calibracion_rgb.calibrar()

# error proyección en px por imagen

In [ ]:
from tabulate import tabulate
tabla=zip(calibracion_rgb.list_nombres_img,calibracion_rgb.list_error_img)
print(tabulate(tabla,headers=["imagen","error en px"]))

# Quitar distorsión a todas las imagenes de la calibración, ver carpeta path/distorsión

In [ ]:
calibracion_rgb.distorsion_imgs()

# Guardar datos calibracion UV para RGB 

In [ ]:
calibracion_rgb.guardar("distorsion_uvz")

# calibración profundidad

In [ ]:

nf=9
nc=6
d=5
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_calibracion/uv_d5_50/"
calibracion_depth=calibracion_uv(path,d,nf,nc,1)

# visualización de las imagenes detectadas y sus puntos

In [ ]:
list_img=calibracion_depth.dibujar_puntos()
for i in range(len(list_img)):
    directorio_img=calibracion_depth.list_nombres_img[i]
    print(directorio_img)
    index=directorio_img.find("depth")
    nombre_img=directorio_img[index:]
    graficar(list_img[i],1,2,[nombre_img,"imagen con puntos"])

# calcular parametros si todas las imagenes fueron correctamente detectados

In [ ]:
calibracion_depth.calibrar()

# error de proyección por imagen

In [ ]:
from  tabulate import tabulate
tabla=zip(calibracion_depth.list_nombres_img,calibracion_depth.list_error_img)
print(tabulate(tabla,headers=["imagen","error en px"]))

# Quitar distorsión a todas las imagenes de la calibración, ver carpeta path/distorsión

In [ ]:
calibracion_depth.distorsion_imgs()

# guardar datos calibracion uv para profundidad

In [ ]:
calibracion_depth.guardar("distorsion_uvz")

# crear clase RGB2DEPTH
1. la clase encuentra la matriz homografica entre los sensores RGB y Profundidad
2. la clase tiene por separado las funciones de resultados y funsiones de visualización.
3.  las imagenes deben estar guardadas con un nombre que posea el siguente formato.  cabecera_numero.png. cabecera=gray indica si la imagen es RGB o cabecera=depth de profunidad. numero puede adquirir cualquier valor.
4. el contructor de la clase resive: path=dirección absoluta del lugar en donde se encuentran las imagenes
5. nf: numero de filas que posee el patron
6. nc: numero de circulos que tiene el patron por fila.
7. para saber que tan buena es la matriz hayada, se encuentra el error promedio de todas las imagenes. el valor promedio es encontrado llevando la informacion RGB del patron al dominio del sensor de profundidad, alli se calculan la posición de los puntos y se los compara con los ayados directamente de la imagen de profundidad
8. los sensores de las imagenes estan fijos, en el caso ideal todas las matrices hograficas hayadas  por cada par de imagenes deberian ser iguales, sin embargo debido al ruido de la escena habra variaciones entre ellas. Para quitar el ruido se se calcula un promedio entre todas las homografias encontradas.

In [ ]:
from calibracion import *
#la clase encuentra la matriz homografica entre los sensores RGB y Profundidad
# la clase tiene por separado las funciones de resultados y funsiones de visualización.
# las imagenes deben estar guardadas con un nombre que posea el siguente formato.  cabecera_numero.png. cabecera=gray indica si la imagen es RGB o cabecera=depth de profunidad. numero puede adquirir cualquier valor.
# el contructor de la clase resive: path=dirección absoluta del lugar en donde se encuentran las imagenes
#nf: numero de filas que posee el patron
#nc: numero de circulos que tiene el patron por fila.calibracion_rgb.mt_inv,
#para saber que tan buena es la matriz hayada, se encuentra el error promedio de todas las imagenes. el valor promedio es encontrado llevando la informacion RGB del patron al dominio del sensor de profundidad, alli se calculan la posición de los puntos y se los compara con los ayados directamente de la imagen de profundidad
# la clase permite guardar la informacion en urchivo .ini. se escoge este formato por dos razones. primero al almacenar la informacion con tiquetas, se  facilita  localizar cualquier dato. se puede editar las etiquetas facilmente, alterando solo la informacion requerita. esto ultimo permite agrupar toda la informacion de configuración en un solo archivo.
# al utilizar el tipo de archivo .ini toda la informacion se guarda en formato string. si bien habrá que reconvertirla a tipo array, se podra abrir el archivo cualquier editor de texto.

class homografia_rgb_depth(puntos):
    def __init__(self,path,nf,nc):
        self.path=path# carpeta direccion absoluta de almacenamiento de las imagenes
        self.nf=nf# numero de filas del patron
        self.nc=nc# numero de circulos por fila.
        self.list_puntos_img=[]# cada elemento estara compunto de 3 matrices. puntos rgb, puntos depth, puntos de imgen rgb en el dominio de depth
        self.list_nombres_img=[]# se almacena solo el  numero (nombre=cabecera_numero) de las imagenes detectadas
        self.homografia=0
        self.error_promedio=0
        self.error_img=[]# se almacena el error promedio de los puntos por cada imagen
         #funciones 
        puntos.__init__(self,nf,nc)# constructor de la clase puntos.
        self.calcular_homografia()
    # abre el par de imagenes de correspondan a la misma escena.
    #devuelve 1 si encuentra el par, 0 caso contrario
    def abrir_par_img(self,indicador1,indicador2,nombre):
        path_1=os.path.join(self.path, '{}_{}.png'.format(indicador1,nombre))
        path_2=os.path.join(self.path, '{}_{}.png'.format(indicador2,nombre))
        #confirmar existencia de ambas imagenes
        ret_2=cv2.haveImageReader(path_2)
        ret_1=cv2.haveImageReader(path_1)
      
        img_1=0;img_2=0;
        ret=ret_1*ret_2#
        if ret:
            img_2=cv2.imread(path_2)
            img_1=cv2.imread(path_1)
           
        return ret,img_1,img_2
    
    def calcular_homografia(self):
        for i in range(1000):
            ret,img_gray,img_depth=self.abrir_par_img("gray","depth",i)
            #print(ret,img_gray,img_depth)
            if ret:
                #encontrar puntos
                ret_gray,puntos_gray,_=self.extraer(img_gray,0)
                ret_depth,puntos_depth,_=self.extraer(img_depth,1)
                ret=ret_depth*ret_gray
                
                print("en imagen numero {} hubo detección de puntos:{}".format(i,bool(ret)))
                if ret:
                    #agregar puntos
                    self.list_puntos_img.append([puntos_gray,puntos_depth,0])
                    #agregar nombre de las imagenes detectadas.
                    self.list_nombres_img.append(i)
                    #encontrar homografia por cada par de imagenes
                    h,ret=cv2.findHomography(puntos_gray, puntos_depth)# puntos fuente, puntos en el destino
                    img_gray_2depth= cv2.warpPerspective(img_gray, h, (img_gray.shape[1], img_gray.shape[0]))#primero ancho y luego alto
                    self.homografia+=h
                
        #eliminar ruido mediante promedio
        self.homografia/=len(self.list_nombres_img)+1
        #convertir todas las imagenes RGB detectadas al dominio de profundidad
        self.convertir_rgb_depth()
    #entrega una lista donde cada elemento tiene 4 imagenes. img_gray: RGB orginal. img_puntos_gray: imagen RGB con puntos detectados. 
    def dibujar_puntos_homografia(self):
        list_img=[]
        for i in range(len(self.list_nombres_img)):
            ret,img_gray,img_depth =self.abrir_par_img("gray","depth",self.list_nombres_img[i])
            img_puntos_gray=self.dibujar(img_gray,self.list_puntos_img[i][0],0)
            img_puntos_depth=self.dibujar(img_depth,self.list_puntos_img[i][1],0)
            list_img.append([img_gray,img_puntos_gray,img_depth,img_puntos_depth])
        return list_img
    #a cada imagen de profundidad se le dibujan los puntos detectados en la RGB con cambio de dominio.
    def dibujar_puntos(self):
        list_img=[]
        for i in range(len(self.list_nombres_img)):
            path_gray_depth=os.path.join(self.path, 'depth_{}.png'.format(self.list_nombres_img[i]))
            img_gray_depth=cv2.imread(path_gray_depth)
            img_puntos_depth=self.dibujar(img_gray_depth.copy(),self.list_puntos_img[i][2],0)
            list_img.append([img_gray_depth,img_puntos_depth])
        return list_img
   #convierte todas las imagenes RGB detectadas al dominio de profundidad. las imagenes son guardadas con nombre de formato gray_2depth_numero. 
    def convertir_rgb_depth(self):
        for i in range(len(self.list_nombres_img)):
            path_gray=os.path.join(self.path, 'gray_{}.png'.format(self.list_nombres_img[i]))
            img_gray=cv2.imread(path_gray)
            #cambiar de dominio
            img_gray_2depth= cv2.warpPerspective(img_gray, self.homografia, (img_gray.shape[1], img_gray.shape[0]))#primero ancho y luego alto
            #extraer puntos en nuevo dominio
            ret,puntos_rgb_depth,_=self.extraer(img_gray_2depth,0)
            #guardar puntos
            self.list_puntos_img[i][2]=puntos_rgb_depth
            if ret:
                #calcular error
                error_img=cv2.norm(self.list_puntos_img[i][1],puntos_rgb_depth, cv2.NORM_L2)/len(puntos_rgb_depth)
                self.error_img.append(error_img)
            else:
                self.error_img.append(5)
            #guardar imagenes 
            path_gray_2depth=os.path.join(self.path, 'gray2_depth_{}.png'.format(self.list_nombres_img[i]))
            cv2.imwrite(path_gray_2depth,img_gray_2depth)
        self.error_img=np.array(self.error_img)
        self.error_promedio=np.mean(self.error_img)
        print("el error de proyección promedio. {}".format(self.error_promedio))
    # guarda en un archivo .ini la informacion de la matriz homografica encontrada y el error de proyecció promedio
    # toda la informacion se coloca en vector columna antes de guardarla
    def guardar(self,nombre,path=""):
        config = ConfigParser()
        #revisar existencia del archivo
        ret=config.read('{}{}.ini'.format(path,nombre))
        if ret:# comprobar objeto vacio
            ret=True
        else:
            ret=False
        config['homografia'] = {'homografia': self.homografia.reshape(-1),
                                   "error":self.error_promedio
                                   }
        #guardar archivo
        with open('{}{}.ini'.format(path,nombre), 'w') as configfile:
            config.write(configfile)
        return ret
        
        
    

In [ ]:
from calibracion import *
rgb_depth=homografia_rgb_depth("/home/estufab4/Desktop/flujo de bagazo/codigo/img_calibracion/uv_d10_110_1/distorsion/",8,5)

# error de proyección promedio en pixeles  por imagenes
1. se ha asignado un error de 5px a las imagenes que no fueron detectadas.

In [ ]:
error=zip(rgb_depth.error_img,rgb_depth.list_nombres_img)
for e in error:
    print(e)


# visualizacion de puntos

In [ ]:
list_img=rgb_depth.dibujar_puntos_homografia()
for i in range(len(list_img)):
    nombre_img=format(rgb_depth.list_nombres_img[i])# colocar numero en formato string
    graficar(list_img[i],2,2,["RGB "+ (nombre_img),"imagen RGB con puntos","profundidad "+(nombre_img),"imagen profundidad con puntos"])

# visualización cambio de dominio

In [ ]:
list_img=rgb_depth.dibujar_puntos()
#print(list_img)
for i in range(len(list_img)):
    nombre_img=format(rgb_depth.list_nombres_img[i])# colocar numero en formato string
   
    graficar(list_img[i],1,2,["imagen depth"+nombre_img,"puntos RGB en depth"])

# guaradar informacion de homografia

In [ ]:
rgb_depth.guardar("configuracion_d10_110_1_p2")

In [ ]:
from calibracion import *
from tabulate import tabulate
nf=8
nc=5
d=10
carpeta="d10_150"
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_calibracion/calibracion2/{}/".format(carpeta)
nombre_config="configuracion_{}".format(carpeta)
def calibracion(nf,nc,d,path,nombre_config):
    #RGB
    calibracion_rgb=calibracion_uv(path,d,nf,nc,0)
    calibracion_rgb.calibrar()
    calibracion_rgb.distorsion_imgs()
    from tabulate import tabulate
    tabla=zip(calibracion_rgb.list_nombres_img,calibracion_rgb.list_error_img)
    print(tabulate(tabla,headers=["imagen","error en px"]))
    calibracion_rgb.guardar(nombre_config)
    #depth
    calibracion_depth=calibracion_uv("{}".format(path),d,nf,nc,1)
    calibracion_depth.calibrar()
    calibracion_depth.distorsion_imgs()
    tabla=zip(calibracion_depth.list_nombres_img,calibracion_depth.list_error_img)
    print(tabulate(tabla,headers=["imagen","error en px"]))
    calibracion_depth.guardar(nombre_config)
    #fusion 
    rgb_depth=homografia_rgb_depth("{}/distorsion/".format(path),nf,nc)
    error=zip(rgb_depth.error_img,rgb_depth.list_nombres_img)
    print(tabulate(error))
    rgb_depth.guardar(nombre_config)
calibracion(nf,nc,d,path,nombre_config)